In [ ]:
import numpy as np
import tensorflow as tf
import io
import binvox_rw
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence

In [ ]:
voxels = Path("models/models-binvox-solid/data")
fvoxels = [f for f in voxels.iterdir() if f.is_file()]

In [ ]:
gen = ImageDataGenerator(rescale=1/255.)
view0 = gen.flow_from_directory(
    "models/models-screenshots/view0",
    target_size=(512, 512),
    seed=1,
    class_mode=None)

view1 = gen.flow_from_directory(
    "models/models-screenshots/view1",
    target_size=(512, 512),
    seed=1,
    class_mode=None)

view2 = gen.flow_from_directory(
    "models/models-screenshots/view2",
    target_size=(512, 512),
    seed=1,
    class_mode=None)

view3 = gen.flow_from_directory(
    "models/models-screenshots/view3",
    target_size=(512, 512),
    seed=1,
    class_mode=None)

In [ ]:
class voxel_gen(Sequence):
    def __init__(self, x_set, batch_size=32, dim=(128,128,128)):
        self.x = x_set # path for each dataset : models/models-binvox-solid
        #self.y = y_set
        self.batch_size = batch_size
        self.dim = dim

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.x) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        x_set_temp = self.x[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        #x_set_temp = [self.x[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(x_set_temp)

        return X, y
    
    def __data_generation(self, x_set_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, 1))
        #y = np.empty((self.batch_size), dtype=int)
        

        # Generate data
        for i, x_set in enumerate(x_set_temp):
            # Store sample
            with open(str(x_set), 'rb') as f:
                model = binvox_rw.read_as_3d_array(f)
            X[i,] = np.expand_dims(model.data, axis=-1)

            # Store class
            #y[i] = self.labels[ID]

        return X, None

In [ ]:
out = voxel_gen(fvoxels)

In [ ]:
def customGen(gen1,gen2,gen3,gen4,gen5):
    while True:
        for (x1,y1),(x2,y2),(x3,y3),(x4,y4),(x5,y5) in zip(gen1,gen2,gen3,gen4,gen5):
            yield ([x1,x2,x3,x4],x5)